In [1]:
import pandas as pd

df = pd.read_csv('hasil_kategori_lda.csv')
df.head()

,Unnamed: 0,id,review,label,pred_single,pred_multiple,terms
0,0,0,i am not necessarily fanatical about this plac...,VALUE,VALUE,VALUE|VALUE|VALUE|VALUE|VALUE|VALUE|VALUE|VALU...,price|place|time|u|rice|
1,1,2,the high prices you're going to pay is for the...,VALUE,VALUE,FOOD|VALUE|VALUE|VALUE|VALUE|VALUE|VALUE|VALUE...,food|high|price|pay|going|view|u|rice|
2,2,3,the high prices you're going to pay is for the...,VALUE,VALUE,FOOD|VALUE|VALUE|VALUE|VALUE|VALUE|VALUE|VALUE...,food|high|price|pay|going|view|u|rice|
3,3,4,the high prices you're going to pay is for the...,VALUE,VALUE,FOOD|VALUE|VALUE|VALUE|VALUE|VALUE|VALUE|VALUE...,food|high|price|pay|going|view|u|rice|
4,4,5,the high prices you're going to pay is for the...,VALUE,VALUE,FOOD|VALUE|VALUE|VALUE|VALUE|VALUE|VALUE|VALUE...,food|high|price|pay|going|view|u|rice|


In [2]:
def calculate(label, is_multi):
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    kosong = 0
    
    for index in range(0, len(df)):
        labels = df['label'][index].split('|')
        if is_multi:
            if type(df['pred_multiple'][index]) != float:
                preds = df['pred_multiple'][index].split('|')

                if label in labels and label in preds:
                    tp += 1
                elif label not in labels and label not in preds:
                    tn += 1
                elif label not in labels and label in preds:
                    fp += 1
                elif label in labels and label not in preds:
                    fn += 1
            elif label in labels:
                fn += 1
            else:
                fp += 1
        else:
            if type(df['pred_single'][index]) != float:
                preds = [df['pred_single'][index]]

                if label in labels and label in preds:
                    tp += 1
                elif label not in labels and label not in preds:
                    tn += 1
                elif label not in labels and label in preds:
                    fp += 1
                elif label in labels and label not in preds:
                    fn += 1
            elif label in labels:
                fn += 1
            else:
                fp += 1
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * ((precision * recall) / (precision + recall))
    print(tp, tn, fp, fn, precision, recall, f1)
    return tp, tn, fp, fn, precision, recall, f1

In [5]:
import pandas as pd

sf = pd.DataFrame(columns=['TP', 'TN', 'FP', 'FN', 'precision', 'recall', 'f1'])

cat = ['AMBIENCE', 'FOOD', 'SERVICE', 'VALUE']
for c in cat:
    a, b, c, d, e, f, g = calculate(c, False)
    sf = sf.append({'TP':a, 'TN':b, 'FP':c, 'FN':d, 'precision':e, 'recall':f, 'f1':g}, ignore_index=True)
sf.to_excel("nikon-false.xlsx")

98 454 39 108 0.7153284671532847 0.47572815533980584 0.5714285714285715
262 272 10 155 0.9632352941176471 0.6282973621103117 0.7605224963715529
11 517 11 160 0.5 0.06432748538011696 0.11398963730569947
81 382 209 27 0.2793103448275862 0.75 0.40703517587939697


In [6]:
import pandas as pd

sf = pd.DataFrame(columns=['TP', 'TN', 'FP', 'FN', 'precision', 'recall', 'f1'])

cat = ['AMBIENCE', 'FOOD', 'SERVICE', 'VALUE']
for c in cat:
    a, b, c, d, e, f, g = calculate(c, True)
    sf = sf.append({'TP':a, 'TN':b, 'FP':c, 'FN':d, 'precision':e, 'recall':f, 'f1':g}, ignore_index=True)
sf.to_excel("nikon-true.xlsx")

169 219 274 37 0.38148984198645597 0.8203883495145631 0.5208012326656395
388 187 95 29 0.8033126293995859 0.9304556354916067 0.8622222222222222
128 499 29 43 0.8152866242038217 0.7485380116959064 0.7804878048780487
108 109 482 0 0.18305084745762712 1.0 0.30945558739255014
